## No need to run again

In [2]:
### pip install the required imports

In [3]:
# #Mount my drive
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!wget https://studio.digitalmaarifa.cloud/train-wav.zip

--2023-11-07 16:52:31--  https://studio.digitalmaarifa.cloud/train-wav.zip
Resolving studio.digitalmaarifa.cloud (studio.digitalmaarifa.cloud)... 188.114.96.3, 188.114.97.3, 2a06:98c1:3121::3, ...
Connecting to studio.digitalmaarifa.cloud (studio.digitalmaarifa.cloud)|188.114.96.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11364410002 (11G) [application/zip]
Saving to: ‘train-wav.zip’

train-wav.zip       100%[===================>]  10.58G  49.6MB/s    in 3m 13s  

2023-11-07 16:55:44 (56.1 MB/s) - ‘train-wav.zip’ saved [11364410002/11364410002]



In [ ]:
!wget https://studio.digitalmaarifa.cloud/train-wav.tsv

In [4]:
!ls /workspace

clips	     onstart.sh  test.tsv   train      train0.tar
onstart.log  ports.log	 test0.tar  train.tsv


In [5]:
! pip install boto3
! pip install botocore
!pip install datasets==1.18.3
!pip install transformers==4.11.3
!pip install huggingface_hub==0.1
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install transformers
! pip install accelerate>=0.20.1
!pip install transformers[torch]

  Using cached transformers-4.11.3-py3-none-any.whl (2.9 MB)
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers
      creating build/lib.linux-x86_64-cpython-310/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers/models
      creating build/lib.linux-x86_64-cpython-310/tokenizers/decod

In [6]:
  ## Start here after restarting notebook

In [7]:
# from huggingface_hub import notebook_login

# notebook_login()

In [8]:
import io
import os
import tarfile
import tempfile
import torch
import re
import json
import boto3
import matplotlib.pyplot as plt
import requests
import torchaudio
from botocore import UNSIGNED
from botocore.config import Config
from IPython.display import Audio
from torchaudio.utils import download_asset
import IPython
from urllib import request
import IPython.display as ipd
import numpy as np
import random
from datasets import Dataset
import pandas as pd
import shutil
import tarfile




from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor


from transformers import TrainingArguments,Trainer


In [9]:
# !mkdir workspace/clips

In [10]:
ls workspace

clips/       onstart.sh  test.tsv   train/     train0.tar
onstart.log  ports.log   test0.tar  train.tsv


In [11]:
# Specify the path to the .tar.gz file
file_path = "/workspace/train0.tar"

# Specify the path to the extraction directory
extract_dir = "/workspace"

# Set the force_extract variable
force_extract = True

# Check if the "clips" folder exists
if os.path.exists(os.path.join(extract_dir, "clips")):
    if force_extract:
        # Delete the "clips" folder and its contents
        shutil.rmtree(os.path.join(extract_dir, "clips"))
        print("Deleted the 'clips' folder.")
    else:
        print("The 'clips' folder already exists. Skipping extraction.")
else:
    print("The 'clips' folder does not exist.")

# Open the .tar.gz file
with tarfile.open(file_path, "r:tar") as tar:
    # Extract all files in the .tar.gz file
    tar.extractall(path=os.path.join(extract_dir, "clips"))

# Get the list of files in the "clips" folder
files = os.listdir(os.path.join(extract_dir, "clips/train"))

# Print the number of files in the "clips" folder
print(f"There are {len(files)} files in the 'clips' folder.")


Deleted the 'clips' folder.
There are 82067 files in the 'clips' folder.


In [12]:
# why is this running ??

In [13]:
# Useful functions

def remove_missing_files(train_tsv_path, clips_train_dir):
    # Read the train.tsv file into a DataFrame
    train_df = pd.read_csv(train_tsv_path, delimiter='\t')

    # Track the number of rows deleted
    rows_deleted = 0

    # Iterate over the rows in the DataFrame
    for index, row in train_df.iterrows():
        audio_file_path = os.path.join(clips_train_dir, os.path.basename(row['path']))
        if not os.path.exists(audio_file_path):
            # Remove the row if the audio file does not exist
            train_df.drop(index, inplace=True)
            rows_deleted += 1

    # Create the final_train.tsv file with the content of train.tsv
    final_train_tsv_path = "final_train.tsv"
    train_df.to_csv(final_train_tsv_path, sep='\t', index=False)

    # Print the number of rows deleted
    print(f"Number of rows deleted: {rows_deleted}")

    return final_train_tsv_path

def plot_waveform(waveform, sample_rate):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    figure.suptitle("waveform")
    plt.show(block=False)


def plot_specgram(waveform, sample_rate, title="Spectrogram"):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
    figure.suptitle(title)
    plt.show(block=False)



chars_to_remove_regex = '[\,\(\)\/\?\.\!\-\;\:\"\“\%\‘\\”\�\'\\’\\ː]'

def remove_special_characters(batch):
    batch = re.sub(chars_to_remove_regex, '', batch).lower()
    return batch

def replace_hatted_characters(batch):
    batch = re.sub('[â]', 'a', batch)
    batch = re.sub('[î]', 'i', batch)
    batch = re.sub('[ô]', 'o', batch)
    batch = re.sub('[û]', 'u', batch)
    batch = re.sub('[μ]', 'u', batch)
    batch = re.sub('[è]', 'e', batch)
    batch = re.sub('[ú]', 'u', batch)
    batch = re.sub('[ó]', 'o', batch)
    batch = re.sub('[í]', 'i', batch)
    batch = re.sub('[é]', 'e', batch)
    return batch


## tokenizer
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}



### obtain audio array
def audio_array(data):
  audio =  [ ]

  for value in  data.path:
    data_1={}
    SAMPLE_MP3 = f"workspace/clips/train/{value}"
    waveform ,sample_rate = torchaudio.load(SAMPLE_MP3)
    #waveform = waveform.to(device)
    if sample_rate != 16000:
        waveform = torchaudio.functional.resample(waveform, sample_rate, 16000)
    data_1["array"] =waveform
    data_1["path"] =f"workspace/clips/train/{value}"
    data_1["sampling_rate"]= 16000
    audio.append(data_1)

  return audio

#testing random data audio
def test_data_audio(data,position=0):
  rand_int = random.randint(0, len(data)-1) if position == 0 else position
  print(data.sentence[rand_int])
  return data.audio[rand_int]["array"].cpu()



def train_test_generator(data):
  train = pd.DataFrame({})
  input_values=[]
  input_length =[]
  labels= []
  data.reset_index(inplace=True)
  for i in range(0,len(data)):
      audio = data["audio"][i]

      # batched output is "un-batched"
      input_values.append(processor(audio["array"].cpu().numpy(), sampling_rate=audio["sampling_rate"]).input_values[0])
      input_length.append(len(input_values[-1]))
      with processor.as_target_processor():
          labels.append(processor(data["sentence"][i]).input_ids)

  train["input_values"] = input_values
  train["input_length"] = input_length
  train["labels"] = labels

  print(train.head())
  # Convert pandas DataFrame to datasets.Dataset
  train = Dataset.from_pandas(train)

  return train



In [14]:
# Reconsile the train.tsv with the audio files

train_tsv_path = "/workspace/train.tsv"
clips_train_dir = "/workspace/clips/train"


new_train_tsv_path = remove_missing_files(train_tsv_path, clips_train_dir)
print(f"New train.tsv file created: {new_train_tsv_path}")

/tmp/ipykernel_5886/1757894689.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_tsv_path, delimiter='\t')


Number of rows deleted: 3
New train.tsv file created: final_train.tsv


In [15]:
# PREPARING THE GPU
print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

2.0.1
2.0.2
cuda


In [16]:
!ls /workspace

clips	     onstart.sh  test.tsv   train      train0.tar
onstart.log  ports.log	 test0.tar  train.tsv


In [17]:
#REAding the data
data = pd.read_csv("final_train.tsv", delimiter='\t').drop(["client_id","up_votes","down_votes","age","gender","accents","locale","segment"],axis=1)#
#test_data = pd.read_csv("/content/corpus/sw/invalidated.tsv", delimiter='\t').drop(["client_id","up_votes","down_votes","age","gender","accents","locale","segment"],axis=1)
print(data.head())

#testing the data
# SPEECH_FILE = download_asset(f"/content/corpus/sw/clips/{data.path[5]}")
# print({data.sentence[5]})
# IPython.display.Audio(SPEECH_FILE)


#Removing unwanted characters
data["sentence"] = data["sentence"].map(lambda x:remove_special_characters(x))
#test_data["sentence"] = test_data["sentence"].map(lambda x:remove_special_characters(str(x)))

# Replacing hatted characters
data["sentence"] = data["sentence"].map(lambda x:replace_hatted_characters(x))
#test_data["sentence"] = test_data["sentence"].map(lambda x:replace_hatted_characters(x))





/tmp/ipykernel_5886/2204190406.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("final_train.tsv", delimiter='\t').drop(["client_id","up_votes","down_votes","age","gender","accents","locale","segment"],axis=1)#


                           path  \
0  common_voice_sw_31428161.mp3   
1  common_voice_sw_30356664.mp3   
2  common_voice_sw_30317714.mp3   
3  common_voice_sw_30055356.mp3   
4  common_voice_sw_29002392.mp3   

                                            sentence variant  
0  wachambuzi wa soka wanamtaja Messi kama nyota ...     NaN  
1  wachambuzi wa soka wanamtaja Messi kama nyota ...     NaN  
2  romario aliingia kwenye orodha ya wachezaji wa...     NaN  
3  romario aliingia kwenye orodha ya wachezaji wa...     NaN  
4                 Inajulikana kama shina la Warangi.     NaN  


In [18]:
#Identifying all characters in the sentence
vocab=[]
for value in data["sentence"]:
  all_text="".join(value)
  vocab = list(set(vocab+list(set(all_text))))

print(vocab)

## Removing " " replacing with tab
vocab_dict = {v: k for k, v in enumerate(sorted(vocab))}
print(vocab_dict)
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

print(vocab_dict)

## adding useful characters
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))
print(vocab_dict)

## save
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

['h', ' ', '9', '`', 'ū', '7', 'g', 'z', '0', 'e', 'k', '\t', '\n', 'i', '_', 'p', 'j', 'd', 'q', 'n', '4', 's', '2', 'f', 'y', 'c', 'w', 'x', 'v', 'o', 'm', '3', '1', '6', 'b', 'l', 'r', 'u', 't', '8', '5', 'a']
{'\t': 0, '\n': 1, ' ': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, '_': 13, '`': 14, 'a': 15, 'b': 16, 'c': 17, 'd': 18, 'e': 19, 'f': 20, 'g': 21, 'h': 22, 'i': 23, 'j': 24, 'k': 25, 'l': 26, 'm': 27, 'n': 28, 'o': 29, 'p': 30, 'q': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'w': 37, 'x': 38, 'y': 39, 'z': 40, 'ū': 41}
{'\t': 0, '\n': 1, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, '_': 13, '`': 14, 'a': 15, 'b': 16, 'c': 17, 'd': 18, 'e': 19, 'f': 20, 'g': 21, 'h': 22, 'i': 23, 'j': 24, 'k': 25, 'l': 26, 'm': 27, 'n': 28, 'o': 29, 'p': 30, 'q': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'w': 37, 'x': 38, 'y': 39, 'z': 40, 'ū': 41, '|': 2}
44
{'\t': 0, '\n': 1, '0': 3, '1': 4, '2': 5,

In [19]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
repo_name = "wav2vec2-kiswahili-tokenizer"

#tokenizer.push_to_hub(repo_name)



Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [20]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [21]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [22]:
# Specify the path to the train.tsv file
train_tsv_path = "/final_train.tsv"

# Specify the path to the "clips/train" directory
clips_train_dir = "/workspace/clips/train"

# Read the train.tsv file into a DataFrame
train_df = pd.read_csv(train_tsv_path, delimiter='\t')

# Print the file path of the first row in the train.tsv file
print("File path in final_train.tsv:")
print(train_df['path'].iloc[20519])

# Get the list of audio files in the "clips/train" directory
audio_files = os.listdir(clips_train_dir)

# Print the name of the first audio file in the "clips/train" directory
print("Audio file name in clips/train:")
print(audio_files[20519])

File path in final_train.tsv:
common_voice_sw_30657069.mp3
Audio file name in clips/train:
common_voice_sw_35586739.mp3


/tmp/ipykernel_5886/4214705235.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_tsv_path, delimiter='\t')


In [23]:
ls

'=0.20.1'                   clips/            lib32@    proc/   tmp/
 NGC-DL-CONTAINER-LICENSE   dev/              lib64@    root/   usr/
 Untitled.ipynb             etc/              libx32@   run/    var/
 Wav2VecCusData1.ipynb      final_train.tsv   media/    sbin@   vocab.json
 bin@                       home/             mnt/      srv/    workspace/
 boot/                      lib@              opt/      sys/


In [24]:
data["audio"] = audio_array(data)
print(data.head())


                           path  \
0  common_voice_sw_31428161.mp3   
1  common_voice_sw_30356664.mp3   
2  common_voice_sw_30317714.mp3   
3  common_voice_sw_30055356.mp3   
4  common_voice_sw_29002392.mp3   

                                            sentence variant  \
0  wachambuzi wa soka wanamtaja messi kama nyota ...     NaN   
1  wachambuzi wa soka wanamtaja messi kama nyota ...     NaN   
2  romario aliingia kwenye orodha ya wachezaji wa...     NaN   
3  romario aliingia kwenye orodha ya wachezaji wa...     NaN   
4                  inajulikana kama shina la warangi     NaN   

                                               audio  
0  {'array': [[tensor(0.), tensor(0.), tensor(0.)...  
1  {'array': [[tensor(3.5646e-12), tensor(6.8450e...  
2  {'array': [[tensor(3.2984e-13), tensor(-6.0890...  
3  {'array': [[tensor(-1.3893e-12), tensor(-7.670...  
4  {'array': [[tensor(8.9046e-14), tensor(4.4616e...  


In [25]:
# test_data["audio"] = audio_array(test_data)
# print(test_data.head())

In [1]:

ipd.Audio(data=test_data_audio(data), autoplay=True, rate=16000)

NameError: name 'ipd' is not defined

In [27]:
from sklearn.model_selection import train_test_split as tts


In [28]:
train_data,valid_data=  tts(data,test_size=0.3)

In [ ]:
train = train_test_generator(train_data)
test = train_test_generator(valid_data)

/opt/conda/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from datasets import  load_metric
wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC


try:
  model = Wav2Vec2ForCTC.from_pretrained(
      "./",
      attention_dropout=0.0,
      hidden_dropout=0.0,
      feat_proj_dropout=0.0,
      mask_time_prob=0.05,
      layerdrop=0.0,
      ctc_loss_reduction="mean",
      pad_token_id=processor.tokenizer.pad_token_id,
      vocab_size=len(processor.tokenizer))

except:
    model = Wav2Vec2ForCTC.from_pretrained(
      "facebook/wav2vec2-xls-r-300m",
      attention_dropout=0.0,
      hidden_dropout=0.0,
      feat_proj_dropout=0.0,
      mask_time_prob=0.05,
      layerdrop=0.0,
      ctc_loss_reduction="mean",
      pad_token_id=processor.tokenizer.pad_token_id,
      vocab_size=len(processor.tokenizer))

In [ ]:
model.freeze_feature_extractor()

In [ ]:
training_args = TrainingArguments(
  output_dir="./wav2vec2-hindi-finetuned0",
  group_by_length=True,
  per_device_train_batch_size=12,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=8,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=100,
  eval_steps=50,
  logging_steps=50,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()

# Testing

In [ ]:
model = trainer.model

In [ ]:
index = 20
input_dict = processor(test[index]["input_values"],sampling_rate=16000, return_tensors="pt", padding=True)
logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]
print("Prediction:")
print(processor.decode(pred_ids))
ipd.Audio(data=test_data_audio(valid_data,index), autoplay=True, rate=16000)

In [ ]:
import json
with open('out.json', 'w') as vocab_file:
    json.dump({"output":f"{pred_ids}"}, vocab_file)